In [1]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


    extract-msg (<=0.29.*)
                 ~~~~~~~^


In [2]:
%pip install glob2

Note: you may need to restart the kernel to use updated packages.


    extract-msg (<=0.29.*)
                 ~~~~~~~^


In [1]:
import pandas as pd
import glob
import os
import re
from pathlib import Path
import logging
from typing import List, Dict, Optional, Tuple

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

print("packages are imported successfully")

packages are imported successfully


In [ ]:
import os 

INPUT_CSV_DIRECTORY = "csv_outputs"          
OUTPUT_DIRECTORY = "cleaned_csv"             
COMBINED_OUTPUT_DIR = "combined_csv"         

# define headers and their variations (coming from the docx extraction)
HEADER_MAPPING = {
    "Week": [
        "Week", "Week No", "Week #", "Week Number", "Week/Date"
    ],
    "Learning Outcomes": [
        "Learning Outcomes", "Learning Outcome", "Learning\\nOutcomes",
        "Learning Outcomes (LO) / Learner and Learning Outcomes (LLO)",
        "Learning Outcomes\\n(At the end of the session, students are expected to:)",
        "Learning Outcomes\\n(At the end of the session, students are expected to :)",
        "Learning\\nOutcomes", "Learning \\nOutcomes"
    ],
    "Deliverables": [
        "Deliverables Outcomes", "Deliverables/Outcomes", "Deliverables", 
        "Deliverables/\\nOutcomes", "Deliverables\\n/ Outcomes", "Deliverables/ Outcomes",
        "Deliverables/\\xa0 Outcomes", "Deliverables/\\xa0\\nOutcomes", 
        "Deliverable / Outcomes", "Deliverables\\n/ Outcomes/Rubrics"
    ],
    "Assessments": [
        "Assessments", "Assessment", "Assessment Task", "Assessment\\n/ Output"
    ]
}

# create output directories 
os.makedirs(OUTPUT_DIRECTORY, exist_ok=True)
os.makedirs(COMBINED_OUTPUT_DIR, exist_ok=True)

print(f"Input directory: {INPUT_CSV_DIRECTORY}")
print(f"Output directory: {OUTPUT_DIRECTORY}")
print(f"Combined output directory: {COMBINED_OUTPUT_DIR}")
print("Configuration completed successfully")

Input directory: csv_outputs
Output directory: cleaned_csv
Combined output directory: combined_csv
Configuration completed successfully


In [3]:
def normalize_header(text):
    """
    Normalize header text: lowercase, remove spaces, slashes, punctuation.
    Same logic as your DOCX extraction script.
    """
    return re.sub(r'[^a-z0-9]', '', text.lower())

def find_csv_files(directory):
    """Find all CSV files in the specified directory."""
    csv_pattern = os.path.join(directory, "*.csv")
    csv_files = glob.glob(csv_pattern)
    return csv_files

def match_headers_to_canonical(df_headers, header_mapping):
    """
    Match DataFrame headers to canonical headers using substring matching.
    Returns matched canonical headers, column indices, and final header names.
    """
    matched_canonical_headers = []
    col_indices = []
    final_headers = []
    
    # Normalize input headers
    normalized_df_headers = [normalize_header(h) for h in df_headers]
    
    for canonical_header, possible_variations in header_mapping.items():
        found_variation_index = -1
        
        # Check if any variation matches as substring
        for variation in possible_variations:
            normalized_variation = normalize_header(variation)
            
            for j, normalized_table_header in enumerate(normalized_df_headers):
                if normalized_variation in normalized_table_header or normalized_table_header in normalized_variation:
                    found_variation_index = j
                    break
                    
            if found_variation_index != -1:
                break
        
        if found_variation_index != -1:
            col_indices.append(found_variation_index)
            final_headers.append(canonical_header)
            matched_canonical_headers.append(canonical_header)
            
    return matched_canonical_headers, col_indices, final_headers

def clean_text_data(df):
    """Clean text data removing common DOCX extraction artifacts."""
    df_clean = df.copy()
    
    for col in df_clean.columns:
        if df_clean[col].dtype == 'object':
            # Strip whitespace
            df_clean[col] = df_clean[col].astype(str).str.strip()
            
            # Replace 'nan' string with empty string
            df_clean[col] = df_clean[col].replace(['nan', 'NaN'], '')
            
            # Clean up common formatting issues from DOCX extraction
            df_clean[col] = df_clean[col].str.replace('\\n', ' ', regex=False)
            df_clean[col] = df_clean[col].str.replace('\\xa0', ' ', regex=False)
            df_clean[col] = df_clean[col].str.replace('  +', ' ', regex=True)  # Multiple spaces to single
    
    return df_clean

print("functions defined successfully")

functions defined successfully


In [ ]:
def process_single_csv(file_path, header_mapping, add_source_info=True):
    """Process a single CSV file and extract target columns."""
    try:
        filename = os.path.basename(file_path)
        print(f"Processing: {filename}")

        # read CSV with encoding handling
        try:
            df = pd.read_csv(file_path, encoding='utf-8')
        except UnicodeDecodeError:
            try:
                df = pd.read_csv(file_path, encoding='latin-1')
                print(f"  Used latin-1 encoding")
            except UnicodeDecodeError:
                df = pd.read_csv(file_path, encoding='cp1252')
                print(f"  Used cp1252 encoding")

        if df.empty:
            print(f"  Warning: Empty file")
            return None # return None for empty files

        print(f"  Original shape: {df.shape}")
        print(f"  Original headers: {list(df.columns)}")

        # match headers to canonical ones
        df_headers = list(df.columns)
        matched_canonical_headers, col_indices, final_headers = match_headers_to_canonical(df_headers, header_mapping)

        # removed the check for len(matched_canonical_headers) < 2
        # the function will now attempt to process even if fewer than 2 headers are matched.

        print(f"  Matched canonical headers: {matched_canonical_headers}")

        # extract the relevant columns
        selected_columns = [df.columns[i] for i in col_indices]
        extracted_df = df[selected_columns].copy()

        # rename columns to canonical names
        column_rename_map = dict(zip(selected_columns, final_headers))
        extracted_df.rename(columns=column_rename_map, inplace=True)

        # add any additional processing steps here
        for canonical_header, variations in header_mapping.items():
            if canonical_header not in extracted_df.columns:
                for col in extracted_df.columns:
                    for variation in variations:
                        if normalize_header(col) == normalize_header(variation):
                            extracted_df[canonical_header] = extracted_df[col]
                            break
                    if canonical_header in extracted_df.columns:
                        break

        # ensure all canonical columns exist and reorder columns - # This part is modified
        # ensure all four canonical columns exist and reorder columns accordingly
        all_canonical_headers = ["Week", "Learning Outcomes", "Deliverables", "Assessments"]
        for canonical_header in all_canonical_headers:
            if canonical_header not in extracted_df.columns:
                extracted_df[canonical_header] = ""

        # reorder columns
        # ensure all four canonical headers are used for reordering
        extracted_df = extracted_df[all_canonical_headers]

        # clean the data
        cleaned_df = clean_text_data(extracted_df)

        # remove completely empty rows
        cleaned_df = cleaned_df.dropna(how='all')

        # ensure all target columns are string type before using .str methods
        for col in ["Week", "Learning Outcomes", "Deliverables", "Assessments"]:
            if col in cleaned_df.columns:
                cleaned_df[col] = cleaned_df[col].astype(str)
                
        # remove rows where all target columns are empty
        mask = (cleaned_df['Week'].str.len() > 0) | \
               (cleaned_df['Learning Outcomes'].str.len() > 0) | \
               (cleaned_df['Deliverables'].str.len() > 0) | \
               (cleaned_df['Assessments'].str.len() > 0)
        cleaned_df = cleaned_df[mask]

        # add source information if requested
        if add_source_info:
            cleaned_df['Source_File'] = filename

            # extract course code from filename
            course_match = re.match(r'^([A-Z]+[0-9]*)', filename)
            if course_match:
                cleaned_df['Course_Code'] = course_match.group(1)
            else:
                cleaned_df['Course_Code'] = ''

        # reset index
        cleaned_df = cleaned_df.reset_index(drop=True)

        print(f"  Final shape: {cleaned_df.shape}")
        print(f"  Successfully processed")

        return cleaned_df

    except Exception as e:
        print(f"  Error processing {os.path.basename(file_path)}: {str(e)}")
        # return None in case of unexpected errors during processing
        return None


# process all CSV files
csv_files = find_csv_files(INPUT_CSV_DIRECTORY)
print(f"Starting to process {len(csv_files)} CSV files...")
print("=" * 60)

processed_files = []
failed_files = []
all_dataframes = []

for file_path in csv_files:
    # pass the HEADER_MAPPING from cell 4 to the processing function
    cleaned_df = process_single_csv(file_path, HEADER_MAPPING, add_source_info=True)

    # check if processing was successful (function did not return None)
    if cleaned_df is not None:
        processed_files.append(file_path)
        # append the DataFrame only if processing was successful and it's not empty
        if not cleaned_df.empty:
             all_dataframes.append(cleaned_df)
             print(f"  Success: {len(cleaned_df)} rows extracted")
        else:
             # file processed successfully but resulted in an empty DataFrame after cleaning
             print(f"  Success: No rows extracted after cleaning")

    else:
        # file processing failed (due to empty file or unexpected error)
        failed_files.append(file_path)
        print(f"  Failed to process")


print("=" * 60)
print(f"  Processing Summary:")
print(f"  Total files: {len(csv_files)}")
print(f"  Successfully processed: {len(processed_files)}")
print(f"  Failed: {len(failed_files)}")
print(f"  Total rows extracted: {sum(len(df) for df in all_dataframes)}")

if failed_files:
    print(f"\nFailed files:")
    for failed_file in failed_files:
        print(f"  - {os.path.basename(failed_file)}")
    
    # a process where in it states what file failed and why
    print("\nheaders:")
    for failed_file in failed_files:
        try:
            df = pd.read_csv(failed_file, nrows=0)
            print(f"  - {os.path.basename(failed_file)} headers: {list(df.columns)}")
        except Exception as e:
            print(f"  - {os.path.basename(failed_file)} error: {e}")

print("\nIndividual processing completed")

Starting to process 57 CSV files...
Processing: AUTOMAT_SYLLABUS_2ndTerm_2024_ver3.0.csv
  Original shape: (10, 4)
  Original headers: ['Week', 'Learning Outcomes', 'Deliverables Outcomes', 'Assessments']
  Matched canonical headers: ['Week', 'Learning Outcomes', 'Deliverables', 'Assessments']
  Final shape: (10, 6)
  Successfully processed
  Success: 10 rows extracted
Processing: CLDCOMP_Syllabus 2023-2024.csv
  Original shape: (11, 4)
  Original headers: ['Week', 'Learning Outcomes', 'Deliverables Outcomes', 'Assessments']
  Matched canonical headers: ['Week', 'Learning Outcomes', 'Deliverables', 'Assessments']
  Final shape: (11, 6)
  Successfully processed
  Success: 11 rows extracted
Processing: CLDSRV2_Syllabus 2023_2024.csv
  Original shape: (5, 4)
  Original headers: ['Week', 'Learning Outcomes', 'Deliverables Outcomes', 'Assessments']
  Matched canonical headers: ['Week', 'Learning Outcomes', 'Deliverables', 'Assessments']
  Final shape: (5, 6)
  Successfully processed
  Succe

In [6]:
def save_individual_files(dataframes_list, original_files_list, output_directory):
    """Save each cleaned DataFrame as an individual CSV file."""
    saved_files = []
    
    print(f"Saving {len(dataframes_list)} cleaned CSV files...")
    print("-" * 40)
    
    for df, original_file_path in zip(dataframes_list, original_files_list):
        try:
            # Create output filename
            original_name = Path(original_file_path).stem
            cleaned_filename = f"{original_name}_cleaned.csv"
            output_path = Path(output_directory) / cleaned_filename
            
            # Save the file
            df.to_csv(output_path, index=False)
            saved_files.append(str(output_path))
            
            print(f"Saved: {cleaned_filename} ({len(df)} rows)")
            
        except Exception as e:
            print(f"Error saving {os.path.basename(original_file_path)}: {str(e)}")
    
    return saved_files

# Save individual cleaned files
if all_dataframes:
    saved_individual_files = save_individual_files(all_dataframes, processed_files, OUTPUT_DIRECTORY)
    
    print("-" * 40)
    print(f"Individual file saving completed")
    print(f"Saved {len(saved_individual_files)} files to: {OUTPUT_DIRECTORY}")
    
    # Show first few saved files
    print(f"\nFirst 5 saved files:")
    for i, saved_file in enumerate(saved_individual_files[:5]):
        print(f"  {i+1}. {os.path.basename(saved_file)}")
    
    if len(saved_individual_files) > 5:
        print(f"  ... and {len(saved_individual_files) - 5} more files")
        
else:
    print("No dataframes to save - check previous steps for errors")

print("\nIndividual file saving completed")

Saving 57 cleaned CSV files...
----------------------------------------
Saved: AUTOMAT_SYLLABUS_2ndTerm_2024_ver3.0_cleaned.csv (10 rows)
Saved: CLDCOMP_Syllabus 2023-2024_cleaned.csv (11 rows)
Saved: CLDSRV2_Syllabus 2023_2024_cleaned.csv (5 rows)
Saved: CLDSRV2_Syllabus_2024_cleaned.csv (10 rows)
Saved: COMPORG_SYLLABUS_AY2024_2025_cleaned.csv (11 rows)
Saved: COMSEC2_Syllabus_2024_cleaned.csv (13 rows)
Saved: COMSEC3_Syllabus_2024 - 2025_cleaned.csv (9 rows)
Saved: COMSECT_Syllabus_2024_cleaned.csv (13 rows)
Saved: Course Syllabus PHYSICS1 Natural Physics 1 for IT_cleaned.csv (10 rows)
Saved: Course Syllabus PHYSICS2 Natural Physics 2 for IT_cleaned.csv (10 rows)
Saved: CRISKMA Syllabus AY 2024 - 2025_cleaned.csv (7 rows)
Saved: CSPROJ2 Course Syllabus AY2024-2025 (KRC)v.03_cleaned.csv (1 rows)
Saved: DASTRUC_Syllabus T1 AY 2024-2025 Revise version_cleaned.csv (11 rows)
Saved: DATAMA1_SYLLABUS_2024_1st_term_Version_cleaned.csv (11 rows)
Saved: DATAMA1_SYLLABUS_Ver3.0_2023_cleaned.cs

In [10]:
def create_combined_dataset(dataframes_list, output_directory, filename="combined_cleaned_syllabi_data.csv"):
    """Combine all cleaned DataFrames into a single CSV file."""
    
    if not dataframes_list:
        print("No dataframes to combine")
        return None
    
    print(f"Creating combined dataset from {len(dataframes_list)} files...")
    
    # Combine all dataframes
    combined_df = pd.concat(dataframes_list, ignore_index=True)
    
    # Sort by source file for better organization
    if 'Source_File' in combined_df.columns:
        combined_df = combined_df.sort_values('Source_File').reset_index(drop=True)
        print("Sorted by source file")
    
    # Create output path
    output_path = Path(output_directory) / filename
    
    # Save combined file
    combined_df.to_csv(output_path, index=False)
    
    print(f"Combined dataset saved to: {output_path}")
    print(f"Total rows in combined dataset: {len(combined_df)}")
    print(f"Columns: {list(combined_df.columns)}")
    
    # Show summary by source file
    if 'Source_File' in combined_df.columns:
        file_counts = combined_df['Source_File'].value_counts()
        print(f"\nRows per source file:")
        print(f"  Total unique files: {len(file_counts)}")
        print(f"  Average rows per file: {file_counts.mean():.1f}")
        print(f"  Min rows per file: {file_counts.min()}")
        print(f"  Max rows per file: {file_counts.max()}")
        
        print(f"\nTop 10 files by row count:")
        for i, (filename, count) in enumerate(file_counts.head(10).items()):
            print(f"  {i+1}. {filename}: {count} rows")
    
    # Show data quality summary
    print(f"\nData quality summary:")
    for col in ['Learning Outcomes', 'Deliverables', 'Assessments']:
        if col in combined_df.columns:
            non_empty = combined_df[col].str.len() > 0
            non_empty_count = non_empty.sum()
            percentage = (non_empty_count / len(combined_df)) * 100
            print(f"  {col}: {non_empty_count}/{len(combined_df)} ({percentage:.1f}%) non-empty")
    
    return str(output_path)

# Create combined dataset
if all_dataframes:
    combined_file_path = create_combined_dataset(
        all_dataframes, 
        COMBINED_OUTPUT_DIR, 
        "combined_cleaned_syllabi_data.csv"
    )
    
    print(f"\nCombined dataset creation completed")
    print(f"File location: {combined_file_path}")
    
else:
    print("No dataframes available for combining")

print("\nCombined dataset ready")

Creating combined dataset from 57 files...
Sorted by source file
Combined dataset saved to: combined_csv\combined_cleaned_syllabi_data.csv
Total rows in combined dataset: 585
Columns: ['Week', 'Learning Outcomes', 'Deliverables', 'Assessments', 'Source_File', 'Course_Code']

Rows per source file:
  Total unique files: 57
  Average rows per file: 10.3
  Min rows per file: 1
  Max rows per file: 19

Top 10 files by row count:
  1. OPESYST_SYLLABUS_AY2024-2025.csv: 19 rows
  2. INPROLA Syllabus T1 A.Y. 2024-2025 Revise version.csv: 18 rows
  3. MOBPROG Course Syllabus 1T 2024-2025 Revise version.csv: 16 rows
  4. MOBPROG Course Syllabus 1T 2022 - Revise Oct 2023.csv: 16 rows
  5. ICTSRV1_Syllabus_2024.csv: 15 rows
  6. PROGCON Syllabus Revision  T1 A.Y. 2023-2024 Revise Oct 2023.csv: 15 rows
  7. SSYADD1-MSYADD1_Syllabus_T1 AY 2024-2025.csv: 14 rows
  8. PROJMAN _Syllabus_T1 AY 2024-2025.csv: 14 rows
  9. MOBAPPL  Syllabus T2 A.Y. 2024-2025 Revise version.csv: 14 rows
  10. INTCOMC_Syllab

In [11]:
def generate_final_summary():
    """Generate a comprehensive summary of the entire process."""
    
    print("=" * 60)
    print("FINAL PROCESSING SUMMARY")
    print("=" * 60)
    
    # Input summary
    print(f"\nINPUT:")
    print(f"  Source directory: {INPUT_CSV_DIRECTORY}")
    csv_files = find_csv_files(INPUT_CSV_DIRECTORY)
    print(f"  Total CSV files found: {len(csv_files)}")
    
    # Processing summary
    print(f"\nPROCESSING RESULTS:")
    print(f"  Successfully processed: {len(processed_files)}")
    print(f"  Failed to process: {len(failed_files)}")
    print(f"  Success rate: {(len(processed_files)/len(csv_files)*100):.1f}%")
    
    if all_dataframes:
        total_rows = sum(len(df) for df in all_dataframes)
        print(f"  Total rows extracted: {total_rows}")
        print(f"  Average rows per file: {total_rows/len(all_dataframes):.1f}")
    
    # Output summary
    print(f"\nOUTPUT:")
    print(f"  Individual cleaned files: {OUTPUT_DIRECTORY}")
    if all_dataframes:
        print(f"    Number of files: {len(all_dataframes)}")
    
    print(f"  Combined dataset: {COMBINED_OUTPUT_DIR}")
    
    # Check if combined file exists and get its info
    combined_file = Path(COMBINED_OUTPUT_DIR) / "combined_cleaned_syllabi_data.csv"
    if combined_file.exists():
        try:
            combined_df = pd.read_csv(combined_file)
            print(f"    Combined file rows: {len(combined_df)}")
            print(f"    Combined file columns: {list(combined_df.columns)}")
        except Exception as e:
            print(f"    Error reading combined file: {e}")
    
    # Failed files details
    if failed_files:
        print(f"\nFAILED FILES:")
        for i, failed_file in enumerate(failed_files, 1):
            print(f"  {i}. {os.path.basename(failed_file)}")
    
    # Header mapping summary
    print(f"\nHEADER MAPPING USED:")
    for canonical, variations in HEADER_MAPPING.items():
        print(f"  {canonical}:")
        for variation in variations[:3]:  # Show first 3 variations
            print(f"    - {variation}")
        if len(variations) > 3:
            print(f"    - ... and {len(variations)-3} more variations")
    
    print("=" * 60)
    print("PROCESS COMPLETED SUCCESSFULLY")
    print("=" * 60)

def validate_output():
    """Validate the output files."""
    
    print("\nVALIDATION:")
    print("-" * 30)
    
    # Check individual files
    individual_files = glob.glob(os.path.join(OUTPUT_DIRECTORY, "*_cleaned.csv"))
    print(f"Individual cleaned files: {len(individual_files)} found")
    
    # Check combined file
    combined_file = Path(COMBINED_OUTPUT_DIR) / "combined_cleaned_syllabi_data.csv"
    if combined_file.exists():
        print(f"Combined file: EXISTS")
        
        # Quick validation of combined file
        try:
            df_combined = pd.read_csv(combined_file)
            print(f"  Shape: {df_combined.shape}")
            
            # Check required columns
            required_cols = ['Learning Outcomes', 'Deliverables', 'Assessments']
            missing_cols = [col for col in required_cols if col not in df_combined.columns]
            
            if missing_cols:
                print(f"  Warning: Missing columns: {missing_cols}")
            else:
                print(f"  All required columns present: {required_cols}")
            
            # Check data completeness
            print(f"  Data completeness:")
            for col in required_cols:
                if col in df_combined.columns:
                    non_empty = (df_combined[col].astype(str).str.len() > 0) & (df_combined[col] != 'nan')
                    count = non_empty.sum()
                    percent = (count / len(df_combined)) * 100
                    print(f"    {col}: {count}/{len(df_combined)} ({percent:.1f}%) non-empty")
            
        except Exception as e:
            print(f"  Error validating combined file: {e}")
    else:
        print(f"Combined file: NOT FOUND")
    
    print("-" * 30)
    print("Validation completed")

# Generate final summary
generate_final_summary()

# Validate output
validate_output()

print("\n" + "=" * 60)
print("ALL STEPS COMPLETED")
print("=" * 60)
print(f"\nYour cleaned data is ready:")
print(f"1. Individual files: {OUTPUT_DIRECTORY}")
print(f"2. Combined file: {COMBINED_OUTPUT_DIR}/combined_cleaned_syllabi_data.csv")

FINAL PROCESSING SUMMARY

INPUT:
  Source directory: csv_outputs
  Total CSV files found: 57

PROCESSING RESULTS:
  Successfully processed: 57
  Failed to process: 0
  Success rate: 100.0%
  Total rows extracted: 585
  Average rows per file: 10.3

OUTPUT:
  Individual cleaned files: cleaned_csv
    Number of files: 57
  Combined dataset: combined_csv
    Combined file rows: 585
    Combined file columns: ['Week', 'Learning Outcomes', 'Deliverables', 'Assessments', 'Source_File', 'Course_Code']

HEADER MAPPING USED:
  Week:
    - Week
    - Week No
    - Week #
    - ... and 2 more variations
  Learning Outcomes:
    - Learning Outcomes
    - Learning Outcome
    - Learning\nOutcomes
    - ... and 5 more variations
  Deliverables:
    - Deliverables Outcomes
    - Deliverables/Outcomes
    - Deliverables
    - ... and 7 more variations
  Assessments:
    - Assessments
    - Assessment
    - Assessment Task
    - ... and 1 more variations
PROCESS COMPLETED SUCCESSFULLY

VALIDATION:
-----